In [2]:
from google.colab import drive
drive.mount("/content/drive")

#from TWSM import *
from tensorflow import keras
from tensorflow.keras import layers


Mounted at /content/drive


In [4]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 46.1 MB/s 
     |████████████████████████████████| 596 kB 39.4 MB/s 
     |████████████████████████████████| 6.6 MB 34.4 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import numpy as np
import pandas as pd
import pickle
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [6]:
DATA_PATH = "/content/drive/Othercomputers/Mi portátil/Master/GitHub/twsm-PERSONAL"
infile = open(DATA_PATH+'/Data/Lemma.pkl','rb')
lemma = pickle.load(infile)

In [7]:
lemma[1]

'clock poll final final clock report acceleration clock upgrade fair number brave soul upgrade clock oscillator share experience poll send brief message detailing experience procedure speed attain cpu rate speed add card adapters heat sink hour usage day floppy disk functionality floppy especially request summarize day add network knowledge base clock upgrade haven answer poll thank guy kuo guykuo washington edu'

In [8]:
df = pd.DataFrame(lemma, columns=["lemma"])
df.head()

,lemma
0,car wonder enlighten car see day door sport ca...
1,clock poll final final clock report accelerati...
2,question folk mac plus finally give ghost week...
3,weitek robert kyanko rob rjck uucp write abrax...
4,shuttle launch question article cowcb world st...


In [9]:
corpus_gen = [doc.split() for doc in df["lemma"]]

# **Part 1: Word2vec**

In [10]:
model = Word2Vec(corpus_gen, size=100, min_count=566)

In [11]:
model.save("word2vec.model")
print(model)

Word2Vec(vocab=429, size=100, alpha=0.025)


In [12]:
model.wv.vocab.keys()

dict_keys(['car', 'wonder', 'see', 'day', 'look', 'early', 'call', 'small', 'body', 'know', 'model', 'year', 'history', 'info', 'mail', 'thank', 'report', 'number', 'experience', 'send', 'message', 'speed', 'rate', 'add', 'card', 'disk', 'request', 'network', 'base', 'answer', 'guy', 'washington', 'edu', 'question', 'mac', 'give', 'start', 'life', 'way', 'new', 'machine', 'bit', 'maybe', 'anybody', 'expect', 'hear', 'suppose', 'access', 'price', 'line', 'like', 'go', 'display', 'probably', 'get', 'feel', 'well', 'great', 'good', 'opinion', 'people', 'use', 'take', 'size', 'money', 'hit', 'real', 'play', 'figure', 'actually', 'advance', 'email', 'post', 'news', 'time', 'truth', 'uucp', 'write', 'article', 'chip', 'far', 'low', 'level', 'stuff', 'pretty', 'require', 'address', 'phone', 'information', 'com', 'thing', 'person', 'sense', 'world', 'clear', 'memory', 'error', 'yes', 'quote', 'software', 'check', 'right', 'value', 'set', 'code', 'tell', 'second', 'idea', 'apr', 'john', 'power'

Word2vec() is creating a model that shows associations between the words of the lemmatized dataframe.

In [13]:
print(model.wv["car"])
print()
model.wv["car"].size

[ 0.25476515 -0.56009346 -0.52613544 -0.11227559 -0.4659684   0.0999354
 -1.8649592   0.14951074 -1.198916   -1.4571494   0.23273368 -0.95715576
 -0.2359499   1.1253877   0.00766562  0.4254204   0.12116097  1.080535
 -0.08702812  0.25497046  0.6104258  -0.26166797 -1.258668   -1.4633013
  1.3461589  -0.887704   -0.6492097  -0.49522713 -0.3339762  -1.2127864
 -0.8016737  -0.45977414 -1.3897207   0.08233285 -0.56338656  0.73973113
 -0.9431021  -0.65727854 -1.4699794  -0.05007841 -0.8803931   1.2428788
 -1.8255843  -0.2199685  -1.095015   -1.1870142   1.1194551  -0.41403282
  0.09069496  1.2858222  -0.07245452 -0.27195194  1.4279633   0.16269441
  0.08171411 -0.3613845   0.787344    1.1858976  -0.675341    0.35646763
 -1.0271226   0.8735899   0.23101923 -0.22533247  0.8711852  -2.1617374
  0.62936234  0.8390513   1.73291     0.04425963  0.24266112 -1.3699682
  0.90033233 -0.373667    0.42513323 -0.09499576 -0.02969022 -0.58676213
 -0.18573545 -0.56551194 -0.6174113   0.6055712   0.0914615

100

The size is due to the 100 we defined on the Word2Vec formula

In [14]:
model.wv.most_similar("car")

[('bike', 0.6204209327697754),
 ('buy', 0.5987908840179443),
 ('friend', 0.5209503173828125),
 ('get', 0.4900822639465332),
 ('hit', 0.48545390367507935),
 ('go', 0.4750940203666687),
 ('price', 0.4722058176994324),
 ('light', 0.46978992223739624),
 ('figure', 0.4572545886039734),
 ('speed', 0.4542411267757416)]

The result shows the most affine words with the word "car"

In [15]:
model.wv.most_similar(positive=['bike', 'machine'], topn=1)

[('stuff', 0.642822802066803)]

In [16]:
model.wv.most_similar(positive=['car', 'buy'], topn=1)

[('price', 0.7206217050552368)]

In [17]:
model.wv.most_similar(positive=['car', 'speed'], topn=1)

[('fast', 0.7158621549606323)]

It's searching for the word with the highest correlation with the other two chosen words 

*Generate a corpus as a list, where for each document the list entry consists of the list of word2vec embeddings of the words in this document. Remove from it the empty documents. Why were those generated? Aggregate the values at a document level (i.e., one embedding per document) by averaging the word embeddings. Store the result for all documents in a data frame and then in GoogleDrive as WordtoVecModel.pkl.*

*Find the most similar words to the embedding representing the first document from 10. Use cosine() to determine the document on the corpus most similar to it.*

# **Part2: Doc2vec**

In [41]:
corpus_gen_td = []
for i, t in enumerate(corpus_gen):
  corpus_gen_td.append(TaggedDocument(t, [i]))

corpus_gen_td[1]

TaggedDocument(words=['clock', 'poll', 'final', 'final', 'clock', 'report', 'acceleration', 'clock', 'upgrade', 'fair', 'number', 'brave', 'soul', 'upgrade', 'clock', 'oscillator', 'share', 'experience', 'poll', 'send', 'brief', 'message', 'detailing', 'experience', 'procedure', 'speed', 'attain', 'cpu', 'rate', 'speed', 'add', 'card', 'adapters', 'heat', 'sink', 'hour', 'usage', 'day', 'floppy', 'disk', 'functionality', 'floppy', 'especially', 'request', 'summarize', 'day', 'add', 'network', 'knowledge', 'base', 'clock', 'upgrade', 'haven', 'answer', 'poll', 'thank', 'guy', 'kuo', 'guykuo', 'washington', 'edu'], tags=[1])

In [21]:
d2v_model = Doc2Vec(corpus_gen_td, vector_size=100, min_count=566)

In [44]:
inf_vec = d2v_model.infer_vector(corpus_gen_td[0][0])

In [46]:
# Emb Representation top10
emb_rep = d2v_model.docvecs.most_similar([inf_vec], topn=10)
emb_rep

[(0, 0.8125115036964417),
 (3550, 0.8087338209152222),
 (1047, 0.7504234313964844),
 (7218, 0.7419774532318115),
 (958, 0.7193106412887573),
 (3819, 0.7128210067749023),
 (596, 0.7003293037414551),
 (8816, 0.6932445764541626),
 (4648, 0.692051887512207),
 (2014, 0.6851178407669067)]

In [42]:
#Most similar to TAG 1
print(d2v_model.docvecs.most_similar(positive=[1], topn=1))

[(11069, 0.6711163520812988)]


# **Part 3: BERT**